In [ ]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph

file_path = "/content/CSO.3.3.ttl"
graph = Graph()
graph.parse(file_path, format="ttl")

def get_superclass(concept):
    query = f"""
      PREFIX ns0: <http://cso.kmi.open.ac.uk/schema/cso#>
      PREFIX ns3: <http://schema.org/>
      SELECT ?subclass ?wikipediaLink
      WHERE {{
          ?subclass ns0:superTopicOf <https://cso.kmi.open.ac.uk/topics/{concept}> .
          ?subclass ns3:relatedLink ?wikipediaLink .
          FILTER(STRSTARTS(STR(?wikipediaLink), "http://en.wikipedia.org/"))
      }}
      """

    results = graph.query(query)
    superclass_info = next(iter(results), None)
    return superclass_info

def get_wikipedia_link(concept):
    query = f"""
      PREFIX ns0: <http://cso.kmi.open.ac.uk/schema/cso#>
      PREFIX ns3: <http://schema.org/>
      SELECT ?wikipediaLink
      WHERE {{
          <https://cso.kmi.open.ac.uk/topics/{concept}> ns3:relatedLink ?wikipediaLink .
          FILTER(STRSTARTS(STR(?wikipediaLink), "http://en.wikipedia.org/"))
      }}
      """

    results = graph.query(query)
    wikipedia_links = [link[0] for link in results]
    return wikipedia_links

def concept_exists(concept):
    query = f"""
        PREFIX ns0: <http://cso.kmi.open.ac.uk/schema/cso#>
        ASK
        WHERE {{
            <https://cso.kmi.open.ac.uk/topics/{concept}> ?predicate ?object .
        }}
    """
    results = graph.query(query)
    return bool(results)

def extract_concept_name(uri):
    return uri.split("/")[-1]

concept = input("Enter the concept: ")
integer_value = int(input("Enter the student ability value: "))

if integer_value > 75:
    if concept_exists(concept):
        word_concept = concept
        wikipedia_links = get_wikipedia_link(word_concept)
    else:
        print("Concept not found in RDF data.")
        exit()
elif 50 <= integer_value <= 75:
    superclass_info = get_superclass(concept)
    if superclass_info:
        word_concept, wikipedia_link = superclass_info
    else:
        print("Superclass not found.")
        exit()
else:
    superclass_info = get_superclass(concept)
    if superclass_info:
        word_concept_uri, wikipedia_link = superclass_info
        if not word_concept_uri:
            print("Superclass of the retrieved superclass not found.")
            exit()
        word_concept = extract_concept_name(word_concept_uri)
        superclass_info = get_superclass(word_concept)
        if superclass_info:
            superclass_uri, wikipedia_link = superclass_info
            if not superclass_uri:
                print("Superclass of the superclass not found.")
                exit()
            word_concept = extract_concept_name(superclass_uri)
            wikipedia_links = get_wikipedia_link(word_concept)
        else:
            print("Superclass of the superclass not found.")
            exit()
    else:
        print("Superclass not found.")
        exit()

# Print the result
print("Resulting concept:", word_concept)
if 50 <= integer_value <= 75:
    if wikipedia_link:
        print("Wikipedia link for the superclass:", wikipedia_link)
    else:
        print("No Wikipedia link found for the superclass.")
else:
    if wikipedia_links:
        print("Wikipedia links for the concept:", wikipedia_links)
    else:
        print("No Wikipedia links found for the concept.")


Enter the concept: distributed_file_system
Enter the student ability value: 45
Resulting concept: operating_system
Wikipedia links for the concept: [rdflib.term.URIRef('http://en.wikipedia.org/wiki/Operating_system')]


In [ ]:
!pip install transformers

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import requests

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

def summarize_wikipedia_content(wikipedia_link):
    # Define the URL for the MediaWiki API
    url = "https://en.wikipedia.org/w/api.php"

    # Define parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": wikipedia_link.split("/")[-1],  # Extract the page title from the URL
        "exintro": True,  # Get only the introductory section
        "explaintext": True,  # Get plaintext instead of HTML
    }

    response = requests.get(url, params=params)

    data = response.json()

    page_id = list(data["query"]["pages"].keys())[0]  # Extract the page ID dynamically
    page_content = data["query"]["pages"][page_id]["extract"]

    inputs = tokenizer([page_content], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=30, max_length=200, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

wikipedia_links='https://en.wikipedia.org/wiki/Operating_system'

summary = summarize_wikipedia_content(wikipedia_links)
print(summary)


An operating system (OS) is system software that manages computer hardware and software resources, and provides common services for computer programs. Operating systems are found on many devices that contain a computer – from cellular phones and video game consoles to web servers and supercomputers.
